In [43]:
# install requests, beautiful soup, and lxml if needed
# import sys
# !conda install --yes --prefix {sys.prefix} requests
# !conda install --yes --prefix {sys.prefix} bs4
# !conda install --yes --prefix {sys.prefix} lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [44]:
# import necessary libraries for retrieving/scraping data
import pandas
import numpy as np
import requests
from bs4 import BeautifulSoup

### Step 1:

a) Load in csv file "teamblind_hiring.csv", which was taken from https://www.teamblind.com/whoshiring. 

b) Use the pandas drop method to get rid of columns that I do not plan to use, i.e. "Where to Apply", "Find on Blind", "Notes". 

c) Sort the DataFrame alphabetically using Company Name. Remove duplicates of company names (remove whole row)

c) Print the first five columns of the dataset

In [45]:
team_blind_data = pandas.read_csv("teamblind_hiring.csv")

team_blind_data = team_blind_data.drop(
    columns=['Where to Apply', 'Find on Blind', 'Notes'])


team_blind_data = team_blind_data.sort_values("Company")
team_blind_data.drop_duplicates(subset="Company", keep='first', inplace=True)

team_blind_data.head()

,Company,Location,Job Function,Date of Submission
182,0x,California,"Software Engineer,Operations,Finance/Accounting",4/23/20 19:20
1216,1010Data,"New York - Chesapeake, VA - Nashville, TN - At...","Business/Strategy,Software Engineer,Sales,Cust...",4/10/20 0:27
183,101edu,New York,Software Engineer,4/23/20 19:20
942,10x Genomics,"Pleasanton, CA / San Francisco, CA / Remote","Software Engineer,Product Manager",4/17/20 16:26
1217,1Password,"US, Canada-Toronto",Software Engineer,4/10/20 0:25


### Step 2:

a) Navigate to the URL: https://github.com/gcreddy42/hiring2020, copy the data under the two tables "Actively Hiring List" & "COVID-19 Summer 2020 Internship Status List", and paste that data into a csv file. 

b) For the table under "Actively Hiring List", remove the columns *Country* and *Position*, remove all rows without the *Job Type* of "Internship", and remove all rows with duplicate company names (this dataset solely focuses on looking at available internship opportunities).

c) Create a new column *Internships* in the csv file and populate the column with respect to availability, i.e. if the company is still hiring interns, then set the cell to **True**, otherwise **False**

*Note*: I did all of the filtering in steps b & c in the csv file itself

d) Read in the csv file and print the first five columns. Drop the "Unnamed" columns as well

In [46]:
github_hiring = pandas.read_csv("github_hiring.csv")
# Removes the last two columns "Unnamed: 2" & "Unnamed: 3" which contained 
# NaN data. Bug most likely originated from filtering in the csv file
github_hiring = github_hiring.drop(columns=["Unnamed: 2", "Unnamed: 3"])
github_hiring.head()

,Company,Internships
0,JPMorgan,True
1,Open Water Accelerator,True
2,TerraClear,True
3,TwinStrand Biosciences,True
4,UserTesting,True


e) Aggregate this dataset with the teamblind dataset to create a new column "Hiring Interns". If a company is in both csv files, and if the company is still hiring interns, i.e. if **True** then fill in "Yes" for that company. If a company is in both csv files, but is not hiring interns anymore, i.e. if **False**, or if a company isn't found in both datasets, then fill in "No" for that company.

f) Print the first five columns of the teamblind dataset. Should have a newly added column, "Hiring Interns"

In [47]:
hiring_interns = []
for company in team_blind_data["Company"]:
    if company in github_hiring["Company"].values:
        company_row = github_hiring.loc[ github_hiring["Company"] == company ]
        hiring_res = company_row["Internships"].iloc[0]
        if hiring_res == True:
            hiring_interns.append("Yes")
        else:
            hiring_interns.append("No")
    else:
        hiring_interns.append("No")

# Adds new column, "Hiring Interns" to team_blind_data DataFrame
team_blind_data["Hiring Interns"] = hiring_interns
team_blind_data.head()

,Company,Location,Job Function,Date of Submission,Hiring Interns
182,0x,California,"Software Engineer,Operations,Finance/Accounting",4/23/20 19:20,No
1216,1010Data,"New York - Chesapeake, VA - Nashville, TN - At...","Business/Strategy,Software Engineer,Sales,Cust...",4/10/20 0:27,No
183,101edu,New York,Software Engineer,4/23/20 19:20,No
942,10x Genomics,"Pleasanton, CA / San Francisco, CA / Remote","Software Engineer,Product Manager",4/17/20 16:26,No
1217,1Password,"US, Canada-Toronto",Software Engineer,4/10/20 0:25,Yes


### Step 3:

a) Import the downloaded csv from Kaggle that contains data about more than 7 million different companies: "companies_sorted.csv)

b) Print the first five columns of the dataset

In [50]:
# 1GB File needed to be loaded - Set to "kaggle_companies" so we don't
# need to load data in every time
kaggle_companies_read = pandas.read_csv("companies_sorted.csv")

In [53]:
kaggle_companies = kaggle_companies_read
kaggle_companies.head()

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,Ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,Tata Consultancy Services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,Accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,Us Army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,Ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960


c) Add the new columns: "Year founded", "Size range", and "Current Employee Estimate" to the teamblind dataset by matching companies found in the downloaded kaggle csv file. Similar to the "Hiring Interns" column, if a company is in both csv files, then fill in the respective value for that company. If there is no match, then set the value to "Unknown" as information for that company under the column is unavailable.

In [54]:
kaggle_companies["name"] = kaggle_companies["name"].str.title()

In [55]:
year_founded = []
size_range = []
curr_employee_est = []
for company in team_blind_data["Company"]:
#     if company in kaggle_companies["name"].values:
        try:
            company_row = kaggle_companies.loc[ kaggle_companies["name"] == company]
            year = company_row["year founded"].iloc[0]
            if np.isnan(year):
                year_founded.append("N/A")
            else:
                year_founded.append(int(year))
            size = company_row["size range"].iloc[0]
            size_range.append(size)
            employee_est = company_row["current employee estimate"].iloc[0].item()
            curr_employee_est.append(employee_est)
        except:
            year_founded.append("Unknown")
            size_range.append("Unknown")
            curr_employee_est.append("Unknown")


In [56]:
team_blind_data["Year Founded"] = year_founded
team_blind_data["Size Range"] = size_range
team_blind_data["Current Employee Estimate"] = curr_employee_est
team_blind_data.head()

,Company,Location,Job Function,Date of Submission,Hiring Interns,Year Founded,Size Range,Current Employee Estimate
182,0x,California,"Software Engineer,Operations,Finance/Accounting",4/23/20 19:20,No,Unknown,Unknown,Unknown
1216,1010Data,"New York - Chesapeake, VA - Nashville, TN - At...","Business/Strategy,Software Engineer,Sales,Cust...",4/10/20 0:27,No,2000,201 - 500,171
183,101edu,New York,Software Engineer,4/23/20 19:20,No,Unknown,Unknown,Unknown
942,10x Genomics,"Pleasanton, CA / San Francisco, CA / Remote","Software Engineer,Product Manager",4/17/20 16:26,No,Unknown,Unknown,Unknown
1217,1Password,"US, Canada-Toronto",Software Engineer,4/10/20 0:25,Yes,2005,11 - 50,23


d) Because there are many "Unknown" values in our dataset currently, remove all rows that contain "Unknown" to help make our data look cleaner

In [57]:
team_blind_data = team_blind_data [ team_blind_data["Year Founded"] != "Unknown"]


### Step 4:

a) Export the team_blind_data DataFrame to a csv file named *final_dataset.csv*. Set index equal to False so we export without the index.

In [58]:
team_blind_data.to_csv("final_dataset.csv", index=False)

### Step 5:

a) There are several founding dates missing for some companies, i.e. N/A. I decided to manually input the founding year through looking up the company.

b) The format of values in the "Location" column differ, so I want to make them all follow the same format. Specifically, only include the state name in which the company is located. For companies that have multiple locations, include all states, and for companies that are remote just denote the value as "Remote". Since there are so many differing formats, I'll be manipulating the data in the CSV file itself. Furthermore, if a company has multiple locations and some of them are out of state, I will filter them out.